In [5]:
import quandl
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn import preprocessing, cross_validation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
quandl.ApiConfig.api_key = 'DEN8cmx6MYrtzyeUdDeh'

In [8]:
%%time
tickers = ['FB','GOOGL','AAPl','AMZN','TSLA','MSFT','AMD','INTC','T','NFLX']
Validate = []
Train = []
for ticker in tickers:
    df = quandl.get("WIKI/"+ticker)
    df = df.iloc[-500:]
    func = ['mean','std','skew','kurt']
    time = [2,4,7,30,60,120,250]
    Z = pd.DataFrame()
    for f in func:
        for t in time:
            aux = pd.DataFrame()
            for i in range(249):
                aux[df.index[i+251]] = list(df.iloc[i-t+251:i+251]['Adj. Close'])
            Z[f+str(t)] = aux.apply(f,axis=0)  
    Z.drop(['skew2','kurt2'],axis=1, inplace =True)
    W = pd.merge(Z, df[['Adj. Close']],left_index=True,right_index=True)
    X = W.drop(['Adj. Close'],1).fillna(0)
    y = W[['Adj. Close']]
    mm1 = MinMaxScaler()
    mm2 = MinMaxScaler()
    mm1.fit(X)
    mm2.fit(y)
    Xm = pd.DataFrame(mm1.transform(X))
    ym = pd.DataFrame(mm2.transform(y))
    Xt, Xv, yt, yv = cross_validation.train_test_split(Xm, ym, test_size = 0.3)
    model =  SGDRegressor()
     
    param_grid = dict(alpha = np.arange(0.0001,0.001),
                      #loss = ['log']
                      penalty =['l2', 'l1', 'elasticnet'],
                      max_iter=[500],
                      epsilon =np.arange(0.0001,0.001),
                      loss = ['squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'])
                      #'perceptron']
                 
    grid = GridSearchCV(cv = 3, 
                        estimator = model,
                        n_jobs = -1,
                        param_grid=param_grid)
    grid.fit(Xm,ym)
    model = grid.best_estimator_
    model.fit(Xt,yt)
    print ticker
    print 'ACC Validate {:.2%}'.format(model.score(Xv,yv))
    print 'ACC Train    {:.2%}'.format(model.score(Xt,yt))
    Validate.append(model.score(Xv,yv))
    Train.append(model.score(Xt,yt))
print 'Promedio Validate  {:.2%}'.format(np.array(Validate).mean())
print 'Promedio Train     {:.2%}'.format(np.array(Train).mean())

/home/carla/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/carla/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/carla/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/carla/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

FB
ACC Validate 95.20%
ACC Train    97.24%
GOOGL
ACC Validate 95.48%
ACC Train    95.63%
AAPl
ACC Validate 95.62%
ACC Train    96.56%
AMZN
ACC Validate 99.52%
ACC Train    98.87%
TSLA
ACC Validate 79.24%
ACC Train    85.75%
MSFT
ACC Validate 98.13%
ACC Train    98.29%
AMD
ACC Validate 79.39%
ACC Train    85.47%
INTC
ACC Validate 97.35%
ACC Train    98.76%
T
ACC Validate 78.88%
ACC Train    89.56%
NFLX
ACC Validate 98.12%
ACC Train    98.50%
Promedio Validate  91.69%
Promedio Train     94.46%
CPU times: user 1min 40s, sys: 1.19 s, total: 1min 41s
Wall time: 2min 12s
